---
layout: post
title: Unit 9 Lesson
description: A progressive journey through Java Inheritance
courses: { csa: {week: 11} }
categories: [C4.0]
type: hacks
---

# Individual Code

## Code Contributions

the feature that i worked on was the chatboard feature in our website. i had wanted to make a chatboard since tri 2 of csp but it was too hard to implement. i was able to implement it in csa because i had more experience with javascript.

---
### [JavaScript (some html and css)](https://github.com/y2kcoders/skatepark.co/blob/main/chatboard.html)
``` html
<html>
<head>
  <h1 class="messing-around">Chat</h1>
  <style>
    /*boring css*/
  </style>
</head>
<body>
  <div class="chatbox">
    <div id="messages">
    </div>
    <div>
      <input type="text" id="input-message" placeholder="Type your message..." />
      <button id="send-button" onclick="sendMessage()" disabled>Send</button>
    </div>
  </div>

  <div class="username-box">
    <input type="text" id="username-input" placeholder="Enter your username...">
    <input type="password" id="password-input" placeholder="Enter your password...">
    <button onclick="login()">Login</button>
  </div>

  <script>
    let loggedIn = false; // Track login status

    function login() {
      const login_url = "https://y2kcoders.stu.nighthawkcodingsociety.com/api/person/";
      const password = document.getElementById("password-input").value;
      const username = document.getElementById("username-input").value;
      fetch(login_url)
        .then(response => response.json())
        .then(data => {
          console.log(data);
          const user = data.find(u => u.name === username && u.password === password);
          if (user) {
            sessionStorage.setItem("uid", user.name);
            loggedIn = true;
            document.getElementById("send-button").removeAttribute("disabled");
            document.getElementById("login-box").style.display = "none";
          } else {
            alert("Username or password is incorrect");
          }
        })
        .catch(error => {
          console.error('Error logging in:', error);
        });
    }


    function updateChat() {
      getMessages();
      setTimeout(updateChat, 2000); // Fetch messages every 2 seconds
    }

    function getMessages() {
      fetch('https://y2kcoders.stu.nighthawkcodingsociety.com/post')
        .then(response => response.json())
        .then(messages => {
          // Reverse the order of messages so the newest is displayed at the top
          messages.reverse();

          const messagesContainer = document.getElementById('messages');
          messagesContainer.innerHTML = ''; // Clear existing messages
          messages.forEach(message => {
            displayMessage(message.writer, message.title, message.content);
          });
        })
        .catch(error => {
          console.error('Error fetching messages:', error);
        });
    }

    function sendMessage() {
      var inputMessage = document.getElementById('input-message').value;
      document.getElementById('input-message').value = '';
      var username = document.getElementById('username-input').value;

      // Get the current time
      var currentTime = getCurrentTime();

      const chatData = {
        "writer": username,
        "title": currentTime,
        "content": inputMessage
      };

      try {
        fetch('https://y2kcoders.stu.nighthawkcodingsociety.com/post', {
          method: 'POST',
          headers: {
            'Content-Type': 'application/json',
          },
          body: JSON.stringify(chatData),
        })
          .then(response => response.json())
          .then(result => {
            if (result.success) {
              // Display the sent message immediately
              displayMessage(username, currentTime, inputMessage);
            }
          })
          .catch(error => {
            console.error('Error:', error);
          });
      } catch (error) {
        console.error('Error:', error);
      }
      getMessages();
    }

    function getCurrentTime() {
      const now = new Date();
      const hours = now.getHours().toString().padStart(2, '0');
      const minutes = now.getMinutes().toString().padStart(2, '0');
      return `${hours}:${minutes}`;
    }

    // Call the updateChat function to start periodic updates
    updateChat();

    function displayMessage(writer, title, content) {
      const messageElement = document.createElement('div');
      messageElement.classList.add('message');
    
      const userElement = document.createElement('span');
      userElement.classList.add('user');
      userElement.textContent = writer + ': ';
    
      // Generate a random color for the user's name
      const randomColor = getRandomColor();
      userElement.style.color = randomColor;
    
      const userMessageElement = document.createElement('span');
      userMessageElement.classList.add('user-message');
      userMessageElement.textContent = content;
    
      const timestampElement = document.createElement('span');
      timestampElement.classList.add('timestamp');
      timestampElement.textContent = title;
    
      messageElement.appendChild(userElement);
      messageElement.appendChild(userMessageElement);
      messageElement.appendChild(timestampElement);
    
      const messagesContainer = document.getElementById('messages');
      messagesContainer.appendChild(messageElement);
    
      // Scroll to the bottom to show the latest messages
      messagesContainer.scrollTop = messagesContainer.scrollHeight;
    }
    
    // Function to generate a random color
    function getRandomColor() {
      const list = ["#FF5733", "#FFC300", "#5DBB63", "#6B66B5", "#4183D7"];
      const randomIndex = Math.floor(Math.random() * list.length);
      return list[randomIndex];
    }
    
  </script>
</body>
</html>
```
---
### [controller file](https://github.com/y2kcoders/backend/blob/master/src/main/java/com/nighthawk/spring_portfolio/mvc/chatboard/PostController.java)
```java
package com.nighthawk.spring_portfolio.mvc.chatboard;

import org.springframework.beans.factory.annotation.Autowired;
import org.springframework.web.bind.annotation.*;

import java.util.List;
import java.util.Optional;

@RestController
public class PostController {

    @Autowired
    PostRepository postRepository;

    @GetMapping("/post")
    public List<Post> getAllPost() {
        return postRepository.findAll();
    }

    @GetMapping("/post/{id}")
    public Post getPost(@PathVariable String id) {
        Long postID = Long.parseLong(id);
        Optional<Post> post = postRepository.findById(postID);
        return post.get();
    }

    @PostMapping("/post/{id}")
    public Post updatePost(@PathVariable String id, @RequestBody Post newPost) {
        Long postID = Long.parseLong(id);
        Optional<Post> post = postRepository.findById(postID);
        post.get().setTitle(newPost.getTitle());
        post.get().setContent(newPost.getContent());
        postRepository.save(post.get());
        return post.get();
    }

    @PostMapping("/post")
    public Post createPost(@RequestBody Post post) {
        // Get the title and content from the request body
        String title = post.getTitle();
        String content = post.getContent();
        String writer = post.getWriter();
        // Create a new Post object
        Post newPost = new Post();
        newPost.setWriter(writer);
        newPost.setTitle(title); // Set the title
        newPost.setContent(content);
        // Set other fields as needed

        // Save the new Post
        postRepository.save(newPost);

        return newPost;
    }

    @DeleteMapping("/post/{id}")
    public String deletePost(@PathVariable String id) {
        Long postID = Long.parseLong(id);
        postRepository.deleteById(postID);
        return "Delete Success!";
    }
}
```
---
### [post.java](https://github.com/y2kcoders/backend/blob/master/src/main/java/com/nighthawk/spring_portfolio/mvc/chatboard/Post.java)
```java
package com.nighthawk.spring_portfolio.mvc.chatboard;
import java.util.Objects;

import javax.persistence.Column;
import javax.persistence.Entity;
import javax.persistence.GeneratedValue;
import javax.persistence.Id;

@Entity
public class Post {

    @Id @GeneratedValue
    @Column(name = "post_id")
    private Long Id;

    private String title;
    private String content;

    private String writer;


    public Long getId() {
        return Id;
    }

    public void setId(Long id) {
        Id = id;
    }

    public String getTitle() {
        return title;
    }

    public void setTitle(String title) {
        this.title = title;
    }

    public String getContent() {
        return content;
    }

    public void setContent(String content) {
        this.content = content;
    }

    public String getWriter() { return writer; }

    public void setWriter(String writer) { this.writer = writer; }

    @Override
    public boolean equals(Object o) {
        if (this == o) return true;
        if (o == null || getClass() != o.getClass()) return false;
        Post post = (Post) o;
        return Objects.equals(Id, post.Id) &&
                Objects.equals(title, post.title) &&
                Objects.equals(content, post.content) &&
                Objects.equals(writer, post.writer);
    }

    @Override
    public int hashCode() {
        return Objects.hash(Id, title, content, writer);
    }
}
```

## collaboration


finn- integrated the log-in feature with the chatboard feature, helped me with the backend

james - was himself, also helped me a lot with the backend

justin - set up the backend for my chatboard to work

## Analytics
### Frontend

<img width="598" alt="Screenshot 2023-11-01 at 1 39 04 PM" src="https://github.com/hsinaDitaM/CSA_Notes/assets/111465192/8333cee5-92ef-4538-be5a-d7053b3dd331">

### Backend

<img width="603" alt="Screenshot 2023-11-01 at 1 40 50 PM" src="https://github.com/hsinaDitaM/CSA_Notes/assets/111465192/460e6198-792d-44e3-a5e9-9a3ec14ec80a">

### frontend commits

#### [Frontend Commits](https://github.com/y2kcoders/skatepark.co/commits?author=hsinaDitaM)

#### [Backend Commits](https://github.com/y2kcoders/backend/commits?author=hsinaDitaM)

# Individual Blog

## Code in blog

<img width="1047" alt="Screenshot 2023-11-01 at 2 10 57 PM" src="https://github.com/hsinaDitaM/CSA_Notes/assets/111465192/68a32043-ddf6-4db1-bccc-534f1b8014a3">

code in blogs can be useful examples. words can always get lost in translation but code will not. 

## Usage Of blog
<img width="284" alt="Screenshot 2023-11-01 at 2 04 35 PM" src="https://github.com/hsinaDitaM/CSA_Notes/assets/111465192/096bd041-8e4e-44db-853f-b7f562fa587f">

I renamed all of the student teaching lessons in order to be more organized so I could find it in the future with the search feature in my site.

<img width="642" alt="Screenshot 2023-11-01 at 2 13 37 PM" src="https://github.com/hsinaDitaM/CSA_Notes/assets/111465192/f39f6ccf-fea5-4b5a-9830-7f1f77cbb67f">

I also used the schedule to organize the blog for future reference. 

## College-board Quiz Notes & test correction

I have not done the test yet but here is the blog that I have set up:

[Link to blog](https://hsinaditam.github.io/CSA_Notes/2023/11/01/test-correction-1.html)

## Student Hacks Grades

| Unit # | Grade          |
|--------|----------------|
| 1      | our lesson     |
| 2      | 0.9            |
| 3      | 0.95           |
| 4      | 0.92           |
| 5      | 0.93           |
| 6      | 0.93           |
| 7      | 0.9            |
| 8      | 0.92           |
| 9      | not graded yet |
| 10     | our lesson     |

## Trimester 1 reflection

Trimester 1 was good trimester. I had a good team and we did everything by the due date. I think we pushed or met the boundaries of what mr mort wanted. one of the good memories that I had from this trimester was our first lesson, I think that went pretty smooth and I learned a lot from it. in the future, I want to further my understanding of java. 




